# Assertion RLA

## Overview of the assertion audit tool

The tool requires as input:

+ audit-specific and contest-specific parameters, such as
    - whether to sample with or without replacement
    - the name of the risk function to use, and any parameters it requires
    - a risk limit for each contest to be audited
    - the social choice function for each contest, including the number of winners
    - candidate identifiers
+ a ballot manifest
+ a random seed
+ a file of cast vote records
+ reported results for each contest
+ json files of assertions for IRV contests (one file per IRV contest)
+ human reading of voter intent from the paper ballots selected for audit

The tool helps select ballots for audit, and reports when the audit has found sufficiently strong evidence to stop.

The tool exports a log of all the audit inputs except the CVR file, but including the auditors' manually determined voter intent from the audited ballots.

In [1]:
from __future__ import division, print_function

import math
import json
import warnings
import numpy as np
import pandas as pd
import csv

from collections import OrderedDict
from IPython.display import display, HTML

from cryptorandom.cryptorandom import SHA256
from cryptorandom.sample import sample_by_index

from assertion_audit_utils import \
    Assertion, Assorter, CVR, TestNonnegMean, check_audit_parameters, write_audit_parameters
from dominion_tools import \
    prep_dominion_manifest, sample_from_manifest, write_ballots_sampled


# Audit parameters.

* `seed`: the numeric seed for the pseudo-random number generator used to draw sample 
* `replacement`: whether to sample with replacement. If the sample is drawn with replacement, gamma must also be specified.
* `risk_function`: the function to be used to measure risk. Options are `kaplan_markov`,`kaplan_wald`,`kaplan_kolmogorov`,`wald_sprt`,`kaplan_martingale`. Not all risk functions work with every social choice function. `wald_sprt` applies only to plurality contests.
* `g`: a parameter to hedge against the possibility of observing a maximum overstatement. Require $g \in [0, 1)$ for `kaplan_markov` and `kaplan_wald`
* `N_ballots`: an upper bound on the number of ballots cast in the contest. This should be derived independently of the voting system.

----

* `cvr_file`: filename for CVRs (input)
* `manifest_file`: filename for ballot manifest (input)
* `assertion_file`: filename of assertions for IRV contests, in RAIRE format
* `mvr_file`: filename for manually ascertained votes from sampled ballots (input)
* `log_file`: filename for audit log (output)

----

* `error_rates`: dict of expected error rates. The keys are
    + `o1_rate`: expected rate of 1-vote overstatements. Recommended value $\ge$ 0.001 if there are hand-marked ballots. Larger values increase the initial sample size, but make it more likely that the audit will conclude in a single round if the audit finds errors
    + `o2_rate`: expected rate of 2-vote overstatements. Recommended value 0.
    + `u1_rate`: expected rate of 1-vote understatements. Recommended value 0.
    + `u2_rate`: expected rate of 2-vote understatements. Recommended value 0.

* `contests`: a dict of contest-specific data 
    + the keys are unique contest identifiers for contests under audit
    + the values are dicts with keys:
        - `risk_limit`: the risk limit for the audit of this contest
        - `ballots_cast`: an upper bound on the number of cast ballots that contain the contest
        - `choice_function`: `plurality`, `supermajority`, or `IRV`
        - `n_winners`: number of winners for majority contests. (Multi-winner IRV not supported; multi-winner super-majority is nonsense)
        - `share_to_win`: for super-majority contests, the fraction of valid votes required to win, e.g., 2/3.
        - `candidates`: list of names or identifiers of candidates
        - `reported_winners` : list of identifier(s) of candidate(s) reported to have won. Length should equal `n_winners`.
        - `assertion_file`: filename for a set of json descriptors of Assertions (see technical documentation) that collectively imply the reported outcome of the contest is correct. Required for IRV; ignored for other social choice functions

In [2]:
seed = 12345678901234567890  # use, e.g., 20 rolls of a 10-sided die. Seed doesn't have to be numeric
replacement = True  # Sampling without replacement isn't implemented
risk_function = "kaplan_martingale"
g=0.1
N_ballots = 300000

In [3]:
cvr_file = './Data/SFDA_2019_Nov8Partial.raire'
manifest_file = './Data/ballot manifest-Sample.xlsx'
mvr_file = './Data/mvr.csv'
log_file = './Data/log.json'

In [4]:
error_rates = {'o1_rate':0.002,      # expect 2 1-vote overstatements per 1000 ballots in the CVR stratum
               'o2_rate':0,          # expect 0 2-vote overstatements
               'u1_rate':0,          # expect 0 1-vote understatements
               'u2_rate':0}          # expect 0 2-vote understatements

In [5]:
# contests to audit. Contest 339 is the DA race
contests = {'339':{'risk_limit':0.05,
                     'choice_function':'IRV',
                     'n_winners':1,
                     'candidates':['15','16','17','18','45'],
                     'reported_winners' : ['15'],
                     'assertion_file' : './Data/SF2019Nov8Assertions.json'
                    }
           }

Example of other social choice functions:

> contests =  {'city_council':{'risk_limit':0.05,
                     'choice_function':'plurality',
                     'n_winners':3,
                     'candidates':['Doug','Emily','Frank','Gail','Harry'],
                     'reported_winners' : ['Doug', 'Emily', 'Frank']
                    },
            'measure_1':{'risk_limit':0.05,
                     'choice_function':'supermajority',
                     'share_to_win':2/3,
                     'n_winners':1,
                     'candidates':['yes','no'],
                     'reported_winners' : ['yes']
                    }                  
           }

## Find audit parameters and conduct audit

* For each contest:
    - find claimed outcome by applying SCF to CVRs
    - complain if claimed outcome disagrees with reported outcome
    - construct assertions that imply contest outcome is correct
    - for each assertion:
        + find generalized diluted margin
        
* Find initial (incremental) sample size from smallest diluted margin, for the sampling plan
    - Complain if expected error rates imply any assertion is incorrect

* For each assertion:
    - Initialize discrepancy counts to zero (o1, o2, u1, u2)
    - Initialize measured risk to 1
* While measured risk for any assertion exceeds its risk limit:
    - expand sample by estimated increment
        + identify ballots in manifest
        + update the log file with incremental sample
    - import audit results when ballots have been audited
    - for each assertion:
        + for each sampled ballot:
            - increment discrepancy count for the assertion
        + find measured risk
    - update log file with new measured risks
    - if any measured risk exceeds its risk limit:
        + estimate incremental sample required to complete the audit

In [6]:
# read the assertions for the IRV contest
for c in contests:
    if contests[c]['choice_function'] == 'IRV':
        with open(contests[c]['assertion_file'], 'r') as f:
            contests[c]['assertion_json'] = json.load(f)['audits'][0]['assertions']

In [7]:
# construct the dict of dicts of assertions for each contest
all_assertions = Assertion.make_all_assertions(contests)

In [8]:
all_assertions

{'339': {'18 v 17 elim 15 16 45': <assertion_audit_utils.Assertion at 0x117296668>,
  '17 v 16 elim 15 18 45': <assertion_audit_utils.Assertion at 0x117296710>,
  '15 v 18 elim 16 17 45': <assertion_audit_utils.Assertion at 0x1172967b8>,
  '18 v 16 elim 15 17 45': <assertion_audit_utils.Assertion at 0x1172961d0>,
  '17 v 16 elim 15 45': <assertion_audit_utils.Assertion at 0x117296320>,
  '15 v 17 elim 16 45': <assertion_audit_utils.Assertion at 0x1172964a8>,
  '15 v 17 elim 16 18 45': <assertion_audit_utils.Assertion at 0x117296438>,
  '18 v 16 elim 15 45': <assertion_audit_utils.Assertion at 0x117296278>,
  '15 v 16 elim 17 45': <assertion_audit_utils.Assertion at 0x1172960f0>,
  '15 v 16 elim 17 18 45': <assertion_audit_utils.Assertion at 0x117296d30>,
  '15 v 16 elim 18 45': <assertion_audit_utils.Assertion at 0x117296f98>,
  '15 v 16 elim 45': <assertion_audit_utils.Assertion at 0x117296f28>,
  '15 v 45': <assertion_audit_utils.Assertion at 0x11727df60>}}

## Read the CVRs 

In [9]:
cvr_input = []
with open(cvr_file) as f:
    cvr_reader = csv.reader(f, delimiter=',', quotechar='"')
    for row in cvr_reader:
        cvr_input.append(row)

print("Read {} rows".format(len(cvr_input)))

Read 118151 rows


In [10]:
cvr_list = CVR.from_raire(cvr_input)
print("After merging, there are CVRs for {} ballots".format(len(cvr_list)))

After merging, there are CVRs for 118149 ballots


In [11]:
# find the mean of the assorters for the CVRs and check whether the assertions are met
assorter_means = {}
min_mean = np.infty
for c in contests.keys():
    contests[c]['cvr_means'] = {}
    for asrtn in all_assertions[c]:
        # find mean of the assertion for the CVRs
        amean = all_assertions[c][asrtn].assorter_mean(cvr_list)
        if amean < 1/2:
            warn("assertion {} not satisfied by CVRs: mean value is {}".format(asrtn, amean))
        contests[c]['cvr_means'][asrtn] = amean
        min_mean = np.min([min_mean, amean])

print("minimum assorter mean {}".format(min_mean))

minimum assorter mean 0.5070715791077368


In [12]:
contests

{'339': {'risk_limit': 0.05,
  'choice_function': 'IRV',
  'n_winners': 1,
  'candidates': ['15', '16', '17', '18', '45'],
  'reported_winners': ['15'],
  'assertion_file': './Data/SF2019Nov8Assertions.json',
  'assertion_json': [{'winner': '18',
    'loser': '17',
    'already_eliminated': ['15', '16', '45'],
    'assertion_type': 'IRV_ELIMINATION',
    'explanation': 'Rules out outcomes with tail [... 18 17]'},
   {'winner': '17',
    'loser': '16',
    'already_eliminated': ['15', '18', '45'],
    'assertion_type': 'IRV_ELIMINATION',
    'explanation': 'Rules out outcomes with tail [... 17 16]'},
   {'winner': '15',
    'loser': '18',
    'already_eliminated': ['16', '17', '45'],
    'assertion_type': 'IRV_ELIMINATION',
    'explanation': 'Rules out outcomes with tail [... 15 18]'},
   {'winner': '18',
    'loser': '16',
    'already_eliminated': ['15', '17', '45'],
    'assertion_type': 'IRV_ELIMINATION',
    'explanation': 'Rules out outcomes with tail [... 18 16]'},
   {'winner':

In [13]:
check_audit_parameters(risk_function, g, error_rates, contests)
write_audit_parameters(log_file, seed, replacement, risk_function, g, N_ballots, error_rates, contests)

## Estimate initial sample size

## Set up for sampling

In [14]:
# read the ballot manifest
# special for SF manifest format
manifest = pd.read_excel(manifest_file)
prep_dominion_manifest(manifest, N_ballots)

/Users/stark/Dropbox/Letters/Ms/Vote/Assertion19/Code/dominion_tools.py:26: UserWarning: Manifest does not account for every ballot cast
  warnings.warn('Manifest does not account for every ballot cast')


In [15]:
manifest

,Tray #,Tabulator Number,Batch Number,Total Ballots,VBMCart.Cart number,cum_ballots
0,1,99808,81,76,3,76
1,1,99808,78,116,3,192
2,1,99808,80,116,3,308
3,1,99808,79,120,3,428
4,1,99808,77,115,3,543
5,2,99804,9,100,2,643
6,2,99804,6,118,2,761
7,2,99804,7,129,2,890
8,2,99804,8,104,2,994
9,2,99804,10,97,2,1091


In [16]:
sample = [0, 60, 1100, 2099]
sam = sample_from_manifest(manifest, sample)
sam

[[3, 1, 99808, 81, 1, '99808-81-1', 1],
 [3, 1, 99808, 81, 61, '99808-81-61', 61],
 [3, 3, 99804, 37, 10, '99804-37-10', 1101],
 [2, 4, 99804, 42, 104, '99804-42-104', 2100]]

In [18]:
write_ballots_sampled('./Data/sample.csv', sam)

In [ ]:
# expand the ballot manifest into a dict. keys are batches, values are ballot numbers.
manifest_parsed = parse_manifest(ballot_manifest)

# check whether manifest accounts for all CVRs and ballots

In [ ]:
# find initial sample size
initial_size = 10 # FIX ME

In [ ]:
m = 78
f = 41
sdm = 4.87
sdf = 6.21
math.sqrt(((m-1)*sdm**2 + (f-1)*sdf**2)/(f+m-2))

In [ ]:
# draw the initial sample
prng = SHA256(seed)
sample = sample_by_index(N_ballots, initial_size, prng=prng)
print(sample)

# Enter the sample data

In [ ]:
# Find audit p-values across assertions
def kaplan_wald(sample, t=1/2, g = 0, random_order = True):
    """
    Kaplan-Wald p-value for the hypothesis that the sample is drawn IID from a nonnegative
    population with mean t against the alternative that the population mean is less than t.
    
    "Pads" the values by g to protect against an observation of 0.
    """
    s = np.array(sample)
    return np.min([1, np.min(np.cumprod((t+g)/(s+g))) if random_order else np.prod((t+g)/(s+g))])

In [ ]:
sample = [1, 1, 1, 1, 1, 0, 0, 0]
kaplan_wald(sample, g=0.01, random_order = False)

In [ ]:
# Identify assertions not yet confirmed

In [ ]:
# Log the status of the audit 

# Escalation: how many more ballots should be drawn?

This tool estimates how many more ballots will need to be audited to confirm any remaining contests. The enlarged sample size is based on:

* ballots already sampled
* assumption that we will continue to see overstatements and understatements at the same rate that observed in the sample

In [ ]:
sample_sizes_new = {}

# TBD
